In [1]:
import cv2
import os
import csv
import numpy as np
import shutil
from skimage.feature import hog
from skimage.morphology import skeletonize
from skimage import exposure

import numpy as np
import tensorflow as tf
from keras import layers, models, Model, Input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# Folder paths
annotation_csv = 'annotations.csv'
dataset_folder = 'dataset'  # Folder containing original images

# Load annotations
with open(annotation_csv, 'r') as file:
    reader = csv.reader(file)
    annotations = list(reader)[1:]  # Skip header
    # annotations = list(reader)[2880:]  # Skip header
    # annotations = list(reader)[1:100]  # Skip header


In [3]:
def test_train_split(image_list, label_list, unique_labels):
    X_train, X_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_state=42)
    label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}  # Map labels to index
    idx_to_label = {idx: label for idx, label in enumerate(unique_labels)}  # Map index to labels
    # Convert labels to numerical indices
    numerical_labels = np.array([label_to_idx[label] for label in label_list])

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(image_list, numerical_labels, test_size=0.1, random_state=42)

    # Normalize the image data (scale pixel values to range [0, 1])
    X_train = np.array(X_train) / 255.0
    X_test = np.array(X_test) / 255.0

    # Convert labels to categorical (one-hot encoding)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(unique_labels))
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(unique_labels))
    X_train = np.expand_dims(X_train, axis=-1)  # (num_samples, height, width, 1)
    X_test = np.expand_dims(X_test, axis=-1)

    # Set input shape based on your image data (e.g., (height, width, channels))
    input_shape = X_train.shape[1:]  # Height, Width, Channels
    return input_shape, X_train, X_test, y_train, y_test


In [4]:
flag = annotations[0][0]
c = 0
image_list1, image_list2, image_list3, image_list4, image_list5 = [], [], [], [], []
label_list = []

# Feature extraction steps
for annotation in annotations:
    c += 1
    image_name, letter, center_x, center_y, dist_x, dist_y = annotation

    # Load original image
    # image_path = os.path.join(dataset_folder, image_name)
    img = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    image_name = image_name[image_name.find('\\')+1:image_name.find('.')]
    # img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"Error: {image_name}_{c} not found!")
        continue

    # Crop the letter using the annotation box
    start_x = int(float(center_x) - float(dist_x) / 2)
    start_y = int(float(center_y) - float(dist_y) / 2)
    end_x = int(float(center_x) + float(dist_x) / 2)
    end_y = int(float(center_y) + float(dist_y) / 2)
    cropped = img[start_y:end_y, start_x:end_x]

    # 1. Image Enhancement & Normalization (Histogram Equalization)
    resized = cv2.resize(cropped, (64, 64))
    enhanced = cv2.equalizeHist(resized)
    norm_image = cv2.normalize(enhanced, None, 0, 255, cv2.NORM_MINMAX)
    image_list1.append(norm_image)
    label_list.append(letter)

    # 2. Segmentation (Thresholding)
    _, segmented = cv2.threshold(norm_image, 30, 255, cv2.THRESH_BINARY_INV)
    image_list2.append(segmented)

    # 3. Edge Detection (Canny)
    edges = cv2.Canny(segmented, 100, 200)
    image_list3.append(edges)

    # 4. Skeletonization
    binary = segmented / 255  # Convert to binary (0, 1)
    skeleton = skeletonize(binary).astype(np.uint8) * 255
    image_list4.append(skeleton)

    # 5. HOG (Histogram of Oriented Gradients)
    hog_features, hog_image = hog(segmented, pixels_per_cell=(4, 4), cells_per_block=(2, 2), visualize=True)
    image_list5.append(hog_image)


In [5]:
#test_train_split(image_list1, label_list1)
#X_train, X_test, y_train, y_test = train_test_split(image_list1, label_list1, test_size=0.2, random_state=42)

In [6]:
# unique_labels = sorted(list(set(label_list1)))  # Get unique classes
# label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}  # Map labels to index
# idx_to_label = {idx: label for idx, label in enumerate(unique_labels)}  # Map index to labels

# # Convert labels to numerical indices
# numerical_labels = np.array([label_to_idx[label] for label in label_list1])

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(image_list1, numerical_labels, test_size=0.1, random_state=42)
# # X_train, X_test = image_list, image_list
# # y_train, y_test = numerical_labels, numerical_labels

# # Normalize the image data (scale pixel values to range [0, 1])
# X_train = np.array(X_train) / 255.0
# X_test = np.array(X_test) / 255.0

# # Convert labels to categorical (one-hot encoding)
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(unique_labels))
# y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(unique_labels))


In [7]:
# Define the CNN model
def cnn_training(input_tensor):
    # model = models.Sequential()
    
    # # First convolutional layer
    # model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    # model.add(layers.MaxPooling2D((2, 2)))
    # # Second convolutional layer
    # model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    # model.add(layers.MaxPooling2D((2, 2)))
    # # Third convolutional layer
    # model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    # model.add(layers.MaxPooling2D((2, 2)))
    # # Flatten the output
    # model.add(layers.Flatten())
    # return model

    # First convolutional layer
    x = layers.Conv2D(32, (3,3), activation = 'relu')(input_tensor)
    x = layers.MaxPooling2D((2,2))(x)

    # Second convolutional layer
    x = layers.Conv2D(64,(3, 3), activation = 'relu')(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Third convolutional layer
    x = layers.Conv2D(64, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Flatten the output
    x = layers.Flatten()(x)
    return x


In [8]:
unique_labels = sorted(list(set(label_list)))  # Get unique classes
num_classes = len(unique_labels)

input_shape1, x_train_1, x_test_1, y_train_1, y_test_1 = test_train_split(image_list1, label_list, unique_labels)
input_shape2, x_train_2, x_test_2, y_train_2, y_test_2 = test_train_split(image_list2, label_list, unique_labels)
input_shape3, x_train_3, x_test_3, y_train_3, y_test_3 = test_train_split(image_list3, label_list, unique_labels)
input_shape4, x_train_4, x_test_4, y_train_4, y_test_4 = test_train_split(image_list4, label_list, unique_labels)
input_shape5, x_train_5, x_test_5, y_train_5, y_test_5 = test_train_split(image_list5, label_list, unique_labels)


In [10]:
# X_train = np.expand_dims(X_train, axis=-1)  # (num_samples, height, width, 1)
# X_test = np.expand_dims(X_test, axis=-1)

# # Set input shape based on your image data (e.g., (height, width, channels))
# input_shape = X_train.shape[1:]  # Height, Width, Channels
# num_classes = len(unique_labels)

input1 = Input(shape = input_shape1)
input2 = Input(shape= input_shape2)
input3 = Input(shape= input_shape3)
input4 = Input(shape= input_shape4)
input5 = Input(shape= input_shape5)

# Create the model
cnn1 = cnn_training(input1)
cnn2 = cnn_training(input2)
cnn3 = cnn_training(input3)
cnn4 = cnn_training(input4)
cnn5 = cnn_training(input5)

concatenated = layers.Concatenate()([cnn1, cnn2, cnn3, cnn4, cnn5])

# x = layers.Concatenate([cnn1, cnn2, cnn3, cnn4, cnn5])
# x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation = 'relu')(concatenated)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs=[input1, input2, input3, input4, input5], outputs = output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit([x_train_1, x_train_2, x_train_3, x_train_4, x_train_5], [y_train_1, y_train_2, y_train_3, y_train_4, y_train_5],
                    epochs = 10, batch_size=32,
                    validation_data=([x_test_1, x_test_2, x_test_3, x_test_4, x_test_5], [y_test_1, y_test_2, y_test_3, y_test_4, y_test_5]))

test_loss, test_acc = model.evaluate([x_test_1, x_test_2, x_test_3, x_test_4, x_test_5],
                                     [y_test_1, y_test_2, y_test_3, y_test_4, y_test_5])
print(f"Test accuracy: {test_acc}")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 62, 62,    │        320 │ input_layer_5[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 62, 62,    │        320 │ input_layer_6[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 62, 62,    │        320 │ input_layer_7[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 62, 62,    │        320 │ input_layer_8[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_27 (Conv2D)  │ (None, 62, 62,    │        320 │ input_layer_9[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_15    │ (None, 31, 31,    │          0 │ conv2d_15[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_18    │ (None, 31, 31,    │          0 │ conv2d_18[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_21    │ (None, 31, 31,    │          0 │ conv2d_21[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_24    │ (None, 31, 31,    │          0 │ conv2d_24[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_27    │ (None, 31, 31,    │          0 │ conv2d_27[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 29, 29,    │     18,496 │ max_pooling2d_15… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 29, 29,    │     18,496 │ max_pooling2d_18

 Total params: 1,764,524 (6.73 MB)

 Trainable params: 1,764,524 (6.73 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 8s 82ms/step - accuracy: 0.1250 - loss: 3.3488 - val_accuracy: 0.3025 - val_loss: 2.3450
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.4361 - loss: 2.1026 - val_accuracy: 0.5617 - val_loss: 1.6196
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.6302 - loss: 1.3497 - val_accuracy: 0.5988 - val_loss: 1.4024
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.7590 - loss: 0.8469 - val_accuracy: 0.6790 - val_loss: 1.2218
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.8777 - loss: 0.4269 - val_accuracy: 0.6358 - val_loss: 1.3741
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.9184 - loss: 0.2357 - val_accuracy: 0.6975 - val_loss: 1.4902
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.9758 - loss: 0.0780 - val_accuracy: 0.6605 - val_loss: 1.8706
Epoch 8/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.9760 - loss: 0.0718 - val_accuracy: 0.7160 - v